In [1]:
# Install packages
!pip install datasets
!pip install transformers==4.28.0

# Clear Colab memory
import os
import shutil
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
try:
    shutil.rmtree('../content/roberta-case')
except FileNotFoundError:
    print('No previous models to remove')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.8 MB/s eta 0:00

In [ ]:
# Import libraries
import torch
import random
import numpy as np
import pandas as pd
from datasets import *
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, EvalPrediction, AutoTokenizer

In [ ]:
def preprocess_data(examples):
    """
    Takes a batch of texts and encodes them as input_ids such that BERT
    can function
    
    Input:
        - examples: Raw data in text form
    
    Output:
        - encoding: Encoded dataset
    """
    # Take a batch of texts and encode them
    text = examples["new_text"]
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    
    # Add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    
    # Fill numpy array with the bach of labels
    labels_matrix = np.zeros((len(text), len(labels)))
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
    encoding["labels"] = labels_matrix.tolist()
    
    return encoding



def multi_label_metrics(predictions, labels, threshold=0.5):
    """
    Computes the f1, roc, and accuracy while training.

    Inputs:
        - predictions: Predictions made by BERT (to be transformed)
        - labels: Labels against which to check the predictions
        - threshold (float): theshold to use for the prediction
    
    Output:
        - metrics (dict): f1, roc_auc, and accuracy
    """
    # Compute predictions from the model
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # Compute f1, roc_auc, and accuracy
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)

    # Build dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics



def compute_metrics(p: EvalPrediction):
    """
    Wrapper to compute the metrics

    Input:
        - p (EvalPrediction): model output
    
    Output:
        - result (dict): f1, roc_auc, and accuracy
    """
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

def select_random_chars(string, max_length=3000):
    """
    BERT cannot process more than 512 tokens so we narrow down
    the text per case to a random window of 3000 characters

    Inputs:
        - string (str): the text of an utterance
        - max_length (int): the maximal length of the new string
    
    Output:
        - selected_chars (str): new, truncated text of an utterance
    """
    string_length = len(string)

    # Check if the string is long enough to select 3000 characters
    if string_length <= max_length:
        return string

    # Generate a random starting index
    start_index = random.randint(0, string_length - max_length)

    # Select the 3000 characters from the string
    selected_chars = string[start_index:start_index + max_length]
    
    return selected_chars

In [ ]:
# Need the CPUs for what comes next
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"torch.device: {device}")

# Get list of files in folder
folder_path = "gdrive/MyDrive/data/"
file_list = os.listdir(folder_path)


# Concatenate the data
df_list = []
for file in file_list:
    # Check if file is a CSV and contains the data we want
    if file.endswith('.csv') and file.startswith('utterances_clean'):
        df = pd.read_csv(os.path.join(folder_path, file))
        df_list.append(df)
df = pd.concat(df_list, axis=0, ignore_index=True)

# For each utterance, add columns specifying who is addressed
df = df.merge(
    df[['speaker', 'side']].drop_duplicates(ignore_index=True),
    how='left',
    left_on='speaker_addressed',
    right_on='speaker',
    suffixes=('', '_addressed')).drop('speaker_addressed', axis=1)

# Translate into natural text (e.g., 'J' --> 'Justice', 'j__clarence_thomas' --> 'Clarence Thomas')
for name_col, type_col in {'speaker': 'speaker_type', 'speaker_replied_to': 'speaker_type_replied_to'}.items():
    df[f'{name_col}_natural'] = df[name_col].apply(lambda name: ' '.join(name.lstrip('j__').split('_')).title())
    speaker_type_translation = {
        'J': 'Justice',
        'A': 'Attorney',
        '<Inaudile>': None
    }

    df[type_col].fillna('na', inplace=True)
    df[f'{type_col}_natural'] = df[type_col].apply(lambda s_type: speaker_type_translation[s_type] if not s_type == 'na' else None)

    df[f'{name_col}_natural'] = df[f'{type_col}_natural'] + ' ' + df[f'{name_col}_natural']
    df[f'{name_col}_natural'].fillna('Unknown', inplace=True)
    df.drop(f'{type_col}_natural', axis=1, inplace=True)

# Also translate the side into natural text (e.g., '1' --> 'Petitioning' (attorney))
side_translation = {
    0: 'Responding',
    1: 'Petitioning',
    2: '',
    3: ''
}

df['side'].fillna(3, inplace=True)
df[f'side_natural'] = df['side'].apply(lambda side: side_translation[side])

df['side_addressed'].fillna(3, inplace=True)
df[f'side_addressed_natural'] = df['side_addressed'].apply(lambda side: side_translation[side])

# Enrich the text of an utterance with some context information
df["new_text"] = "<UTTERANCE_START>" + df['side_natural'] + " " + df["speaker_natural"] + " says: '" + df["text"] + "' to " + df['side_addressed_natural'] + " " + df["speaker_replied_to_natural"] + " <UTTERANCE_END>"



<ipython-input-5-739b003cceb3>:12: DtypeWarning: Columns (17,56,58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-5-739b003cceb3>:12: DtypeWarning: Columns (17,60,62,64,66,68,70,72,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-5-739b003cceb3>:12: DtypeWarning: Columns (58,60,62,64,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-5-739b003cceb3>:12: DtypeWarning: Columns (58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))


In [ ]:
# Choose justices
j_columns = ['votes_side_j__ruth_bader_ginsburg', 'votes_side_j__clarence_thomas', 'votes_side_j__stephen_g_breyer', 'votes_side_j__anthony_m_kennedy', 'votes_side_j__antonin_scalia', 'votes_side_j__john_g_roberts_jr', 'votes_side_j__samuel_a_alito_jr', 'votes_side_j__john_paul_stevens', 'votes_side_j__sonia_sotomayor', 'votes_side_j__david_h_souter', 'votes_side_j__elena_kagan', 'votes_side_j__sandra_day_oconnor', 'votes_side_j__william_h_rehnquist', 'votes_side_j__neil_gorsuch', 'votes_side_j__brett_m_kavanaugh', 'win_side']


In [ ]:

results = pd.DataFrame()

for justice in j_columns:

  # Data Wrangling for each justice
  df_j = df[['case_id', 'new_text'] + [justice]]
  grouped_df = df_j.groupby('case_id')['new_text'].apply(lambda x: ','.join(x)).reset_index()
  justices = df[["case_id"] + [justice]].drop_duplicates(keep='first')

  df1 = pd.merge(grouped_df, justices, left_on='case_id', right_on='case_id', how='left').dropna(axis='rows', how='any')
  df1 = df1.drop(df1[~df1[justice].isin([0, 1])].index)

  df1['new_text'] = df1['new_text'].apply(select_random_chars)

  dataset = Dataset.from_pandas(df1.drop('case_id', axis=1), preserve_index = False)

  dataset = dataset.train_test_split(test_size=0.3, shuffle=True)

  dataset_test_valid = dataset['test'].train_test_split(test_size=0.5, shuffle=True)

  dataset = DatasetDict({
      'train': dataset['train'],
      'test': dataset_test_valid['test'],
      'validation': dataset_test_valid['train']})

  labels = [label for label in dataset['train'].features.keys() if label not in ['case_id', 'new_text']]
  id2label = {idx:label for idx, label in enumerate(labels)}
  label2id = {label:idx for idx, label in enumerate(labels)}
  
  # Start creating model
  tokenizer = AutoTokenizer.from_pretrained('roberta-base', return_overflowing_tokens=True)

  encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

  model = AutoModelForSequenceClassification.from_pretrained('roberta-base', 
                                                            problem_type="multi_label_classification", 
                                                            num_labels=len(labels),
                                                            id2label=id2label,
                                                            label2id=label2id)

  batch_size = 8
  metric_name = "f1"

  args = TrainingArguments(
      "test-trainer",
      evaluation_strategy = "epoch",
      save_total_limit = 2,
      save_strategy = "epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=15,
      weight_decay=0.01,
      load_best_model_at_end=True,
      metric_for_best_model=metric_name,
      #push_to_hub=True,
  )

  trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
  )
  trainer.train()

  val_metrics = trainer.evaluate()
  val_metrics = dict((key.replace('eval', 'val'), [value]) for (key, value) in val_metrics.items())

  # Initialize a new trainer instance with the trained model and test data
  trainer_test = Trainer(
      model=model,  
      args=args,
      eval_dataset=encoded_dataset["test"], 
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
      )

  # Evaluate the model on the test data
  test_metrics = trainer_test.evaluate()
  test_metrics = dict((key.replace('eval', 'test'), [value]) for (key, value) in test_metrics.items())

  metrics = val_metrics
  metrics.update(test_metrics)
  metrics['justice'] = [justice]

  results = pd.concat([results, pd.DataFrame(metrics)], ignore_index=True)

results = results[['justice', 'epoch'] +[col for col in results.columns if col.startswith('val_')] +
    [col for col in results.columns if col.startswith('test_')]]



1.0    0.584173
0.0    0.415827
Name: votes_side_j__ruth_bader_ginsburg, dtype: float64


Map:   0%|          | 0/973 [00:00<?, ? examples/s]

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

...

Map:   0%|          | 0/980 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.625372,0.704762,0.500000,0.704762
2,No log,0.625446,0.704762,0.500000,0.704762
3,No log,0.613686,0.704762,0.500000,0.704762
4,No log,0.630181,0.704762,0.500000,0.704762
5,0.660400,0.730334,0.561905,0.581408,0.561905
6,0.660400,0.775058,0.633333,0.524303,0.633333
7,0.660400,1.385256,0.647619,0.539124,0.647619
8,0.660400,1.591407,0.652381,0.519071,0.652381
9,0.409100,2.081824,0.619048,0.570401,0.619048
10,0.409100,2.142974,0.633333,0.528989,0.633333


In [ ]:
results


,justice,epoch,val_loss,val_f1,val_roc_auc,val_accuracy,val_runtime,val_samples_per_second,val_steps_per_second,test_loss,test_f1,test_roc_auc,test_accuracy,test_runtime,test_samples_per_second,test_steps_per_second
0,votes_side_j__ruth_bader_ginsburg,15.0,0.673158,0.600962,0.500000,0.600962,1.6294,127.654,15.957,0.679074,0.583732,0.500000,0.583732,1.5806,132.226,17.082
1,votes_side_j__clarence_thomas,15.0,0.683534,0.576923,0.500000,0.576923,1.6136,128.902,16.113,0.668339,0.617225,0.500000,0.617225,1.5732,132.849,17.162
2,votes_side_j__stephen_g_breyer,15.0,0.632885,0.684466,0.500000,0.684466,1.6245,126.810,16.005,0.645785,0.657005,0.500000,0.657005,1.5372,134.663,16.914
3,votes_side_j__anthony_m_kennedy,15.0,0.651891,0.657895,0.500000,0.657895,1.4871,127.764,16.139,0.638538,0.689474,0.500000,0.689474,1.4126,134.507,16.990
4,votes_side_j__antonin_scalia,15.0,0.680410,0.588957,0.500000,0.588957,1.2805,127.292,16.400,0.645863,0.658537,0.500000,0.658537,1.2528,130.909,16.763
5,votes_side_j__john_g_roberts_jr,15.0,0.623760,0.686667,0.500000,0.686667,1.1629,128.988,16.338,0.691134,0.582781,0.500000,0.582781,1.1383,132.654,16.692
6,votes_side_j__samuel_a_alito_jr,15.0,0.687771,0.579310,0.500000,0.579310,1.1363,127.605,16.721,0.680201,0.643836,0.500000,0.643836,1.1050,132.128,17.195
7,votes_side_j__john_paul_stevens,15.0,0.835364,0.558559,0.562013,0.558559,0.9049,122.665,15.471,0.905748,0.517857,0.533844,0.517857,0.8515,131.532,16.442
8,votes_side_j__sonia_sotomayor,15.0,0.664657,0.650943,0.500000,0.650943,0.8420,125.896,16.628,0.665082,0.654206,0.500000,0.654206,0.8060,132.759,17.370
9,votes_side_j__david_h_souter,15.0,0.683548,0.580000,0.500000,0.580000,0.7915,126.342,16.425,0.668417,0.680000,0.500000,0.680000,0.7476,133.759,17.389


In [ ]:
print(df1)